In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn.functional as F
from transformers import RobertaTokenizer, RobertaConfig, RobertaForSequenceClassification,get_linear_schedule_with_warmup
# Import AdamW from torch.optim
from torch.optim import AdamW


import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
# Import and evaluate each test batch using Matthew's correlation coefficient
from sklearn.metrics import accuracy_score,matthews_corrcoef

from tqdm import tqdm, trange,tnrange,tqdm_notebook
import random
import os
import io
# %matplotlib inline # Remove or comment out this line if not in a Jupyter Notebook
# Instead, use plt.show() after creating your plots
# For example:
# plt.plot([1, 2, 3], [4, 5, 6])
# plt.show()

**ROBERTA - X-Twitter Sentiment Classifier**

Identify and specify the GPU as the device, later in training loop we will load data into device

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn.functional as F
from transformers import RobertaTokenizer, RobertaConfig, RobertaForSequenceClassification,get_linear_schedule_with_warmup
# Import AdamW from torch.optim
from torch.optim import AdamW


import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
# Import and evaluate each test batch using Matthew's correlation coefficient
from sklearn.metrics import accuracy_score,matthews_corrcoef

from tqdm import tqdm, trange,tnrange,tqdm_notebook
import random
import os
import io
# %matplotlib inline # Remove or comment out this line if not in a Jupyter Notebook
# Instead, use plt.show() after creating your plots
# For example:
# plt.plot([1, 2, 3], [4, 5, 6])
# plt.show()

# identify and specify the GPU as the device, later in training loop we will load data into device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

# Check if CUDA is available before trying to get device name
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
else:
    print("CUDA is not available. Using CPU instead.")

SEED = 19

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if device == torch.device("cuda"):
    torch.cuda.manual_seed_all(SEED)

**Read File**

In [ ]:
# Assuming the CSV file is in the same directory as your script
df_train = pd.read_csv("Twitter_Data.csv")  # Remove the incorrect path

# OR, if the file is in a different location, use the correct path
# df_train = pd.read_csv("/path/to/your/file/Twitter_Data.csv") # Replace with your correct path

In [ ]:
df_train.isnull().sum()

In [ ]:
df_train.head()

In [ ]:
df_train['category'].unique()

In [ ]:
df_train['category'].value_counts()

In [ ]:
df_train = df_train[~df_train['category'].isnull()]

In [ ]:
df_train = df_train[~df_train['clean_text'].isnull()]

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
df_train['category_1'] = labelencoder.fit_transform(df_train['category'])

In [ ]:
df_train[['category','category_1']].drop_duplicates(keep='first')

In [ ]:
df_train.rename(columns={'category_1':'label'},inplace=True)

In [ ]:
## create label and sentence list
sentences = df_train.clean_text.values

#check distribution of data based on labels
print("Distribution of data based on labels: ",df_train.label.value_counts())

# Set the maximum sequence length. The longest sequence in our training set is 47, but we'll leave room on the end anyway.
# In the original paper, the authors used a length of 512.
MAX_LEN = 256

## Import ROBERTA tokenizer, that is used to convert our text into tokens that corresponds to ROBERTA library
tokenizer = RobertaTokenizer.from_pretrained('roberta-base',do_lower_case=True)

In [ ]:
input_ids = [tokenizer.encode(sent, add_special_tokens=True,
                              max_length=MAX_LEN,
                              pad_to_max_length=True,truncation=True) for sent in sentences]

In [ ]:
labels = df_train.label.values

print("Actual sentence before tokenization: ",sentences[2])
print("Encoded Input from dataset: ",input_ids[2])

## Create attention mask
attention_masks = []
## Create a mask of 1 for all input tokens and 0 for all padding tokens
attention_masks = [[float(i>0) for i in seq] for seq in input_ids]
print(attention_masks[2])

In [ ]:
train_inputs,validation_inputs,train_labels,validation_labels = train_test_split(input_ids,labels,random_state=41,test_size=0.1)
train_masks,validation_masks,_,_ = train_test_split(attention_masks,input_ids,random_state=41,test_size=0.1)

In [ ]:
# convert all our data into torch tensors, required data type for our model
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

# Select a batch size for training. For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32
batch_size = 32

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop,
# with an iterator the entire dataset does not need to be loaded into memory
train_data = TensorDataset(train_inputs,train_masks,train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data,sampler=train_sampler,batch_size=batch_size)

validation_data = TensorDataset(validation_inputs,validation_masks,validation_labels)
validation_sampler = RandomSampler(validation_data)
validation_dataloader = DataLoader(validation_data,sampler=validation_sampler,batch_size=batch_size)

In [ ]:
train_data[0]

In [ ]:
from transformers import RobertaForSequenceClassification, get_linear_schedule_with_warmup
from torch.optim import AdamW

# Load RoBERTa model with classification head
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3).to(device)

# Training hyperparameters
lr = 2e-5
adam_epsilon = 1e-8
epochs = 3

# Calculate training steps
num_training_steps = len(train_dataloader) * epochs
num_warmup_steps = 0

# Optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=lr, eps=adam_epsilon)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)

# Reset gradients
model.zero_grad()



In [ ]:
# Import necessary libraries
import torch
from transformers import RobertaForSequenceClassification, get_linear_schedule_with_warmup
from torch.optim import AdamW
import psutil


In [ ]:
# Function to print memory usage
def print_memory_usage():
    mem = psutil.virtual_memory()
    print(f"Used: {mem.used / 1e9:.2f} GB / Total: {mem.total / 1e9:.2f} GB")

# Print memory before model loading
print("Before model load:")
print_memory_usage()


In [ ]:
# Load RoBERTa model on CPU
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3).to('cpu')


In [ ]:
# Print memory after model load
print("After model load:")
print_memory_usage()


In [ ]:
# Set training hyperparameters
lr = 2e-5
adam_epsilon = 1e-8
epochs = 3

# Number of training steps
num_training_steps = len(train_dataloader) * epochs  # Ensure train_dataloader is defined
num_warmup_steps = 0

# Optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=lr, eps=adam_epsilon)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)

# Reset gradients
model.zero_grad()


In [ ]:
!ls

In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
model = RobertaForSequenceClassification.from_pretrained('distilroberta-base', num_labels=3).to('cpu')


In [ ]:
## Store our loss and accuracy for plotting
train_loss_set = []
learning_rate = []

# Gradients gets accumulated by default
model.zero_grad()

# tnrange is a tqdm wrapper around the normal python range
for _ in tnrange(1,epochs+1,desc='Epoch'):
  print("<" + "="*22 + F" Epoch {_} "+ "="*22 + ">")
  # Calculate total loss for this epoch
  batch_loss = 0

  for step, batch in enumerate(train_dataloader):
    # Set our model to training mode (as opposed to evaluation mode)
    model.train()

    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch

    # Forward pass
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    loss = outputs[0]

    # Backward pass
    loss.backward()

    # Clip the norm of the gradients to 1.0
    # Gradient clipping is not in AdamW anymore
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    # Update parameters and take a step using the computed gradient
    optimizer.step()

    # Update learning rate schedule
    scheduler.step()

    # Clear the previous accumulated gradients
    optimizer.zero_grad()

    # Update tracking variables
    batch_loss += loss.item()

  # Calculate the average loss over the training data.
  avg_train_loss = batch_loss / len(train_dataloader)

  #store the current learning rate
  for param_group in optimizer.param_groups:
    print("\n\tCurrent Learning rate: ",param_group['lr'])
    learning_rate.append(param_group['lr'])

  train_loss_set.append(avg_train_loss)
  print(F'\n\tAverage Training loss: {avg_train_loss}')

  # Validation

  # Put model in evaluation mode to evaluate loss on the validation set
  model.eval()

  # Tracking variables
  eval_accuracy,eval_mcc_accuracy,nb_eval_steps = 0, 0, 0

  # Evaluate data for one epoch
  for batch in validation_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

    # Move logits and labels to CPU
    logits = logits[0].to('cpu').numpy()
    label_ids = b_labels.to('cpu').numpy()

    pred_flat = np.argmax(logits, axis=1).flatten()
    labels_flat = label_ids.flatten()

    df_metrics=pd.DataFrame({'Epoch':epochs,'Actual_class':labels_flat,'Predicted_class':pred_flat})

    tmp_eval_accuracy = accuracy_score(labels_flat,pred_flat)
    tmp_eval_mcc_accuracy = matthews_corrcoef(labels_flat, pred_flat)

    eval_accuracy += tmp_eval_accuracy
    eval_mcc_accuracy += tmp_eval_mcc_accuracy
    nb_eval_steps += 1

  print(F'\n\tValidation Accuracy: {eval_accuracy/nb_eval_steps}')
  print(F'\n\tValidation MCC Accuracy: {eval_mcc_accuracy/nb_eval_steps}')

In [ ]:
## emotion labels
label2int = {
  "Negative": 0,
  "Neutral": 1,
  "Positive": 2
}

In [ ]:
print(classification_report(df_metrics['Actual_class'].values, df_metrics['Predicted_class'].values, target_names=label2int.keys(), digits=len(label2int)))

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    import itertools
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
confusion_matrix(df_metrics['Actual_class'].values, df_metrics['Predicted_class'].values)

In [ ]:
plot_confusion_matrix(cm=confusion_matrix(df_metrics['Actual_class'].values, df_metrics['Predicted_class'].values),
                      classes=[0  ,1  ,2],
                          normalize=True,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues)